In [0]:
dbutils.widgets.text("data_execucao", "")
data_execucao = dbutils.widgets.get("data_execucao")

In [0]:
import requests
from pyspark.sql.functions import lit

In [0]:
def extract_data(date, base="BRL"):

  url = f"https://api.apilayer.com/exchangerates_data/{date}&base={base}"

  headers = {
  "apikey": "6S38Kwmfck6QmL6BLSD4JmHBK8V1cGxP"
 }

  parametros = {"base":base}
 
  response = requests.request("GET", url, headers=headers, params = parametros)

  if response.status_code != 200:
    raise Exception(f"Não foi possivel extrair os dados!!! | status_code {response.status_code}")

  return response.json()

In [0]:
def data_to_df (data_json): 
    
    data_tuple = [(moeda, float (taxa)) for moeda, taxa in data_json["rates"].items()]
    return data_tuple

In [0]:
def save_to_parquet(converted_data):

  ano, mes, dia = converted_data['date'].split('-')
  full_path = f"dbfs:/databricks-results/bronze/{ano}/{mes}/{dia}/"

  response = data_to_df(converted_data)
  df = spark.createDataFrame(response, ["moeda", "taxa"])
  df = df.withColumn('data', lit(f"{ano}-{mes}-{dia}"))

  df.write.mode("overwrite").format("parquet").save(full_path)

  print(f"Os arquivos foram salvos em {full_path}")

In [0]:
cotas = extract_data(data_execucao)
save_to_parquet(cotas)